# Projeto 1 - Ciência dos Dados

Nome: João Vitor Costa Alves

Nome: Felipe Avila Braun

___
### Atenção: <br> $\quad$* Serão permitidos grupos de até 3 pessoas, mas com uma rubrica mais exigente. <br> $\quad$* Veja RUBRICA na última página do arquivo PDF que traz o enunciado do Proejto 1.

___
Carregando algumas bibliotecas:

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\jvca0\Documents\Code\cDados\p1_cdados


___

### Carregando a base de dados com as mensagens dos seus arquivos. <br> Tire o `#` do início da linha de código condizente ao caso escolhido para o projeto 1.

In [6]:
nome = input("Digite o username digitado quando arquivo foi criado: ")

In [7]:
# SE SEU PROJETO USA OS DADOS SOBRE "Airline Passenger Reviews"
#train = pd.read_csv('dados_treino_ate_TRIO_'+nome+'.csv')
#test = pd.read_csv('dados_teste_ate_TRIO_'+nome+'.csv')

In [10]:
#SE SEU PROJETO USA OS DADOS SOBRE "ChatGPT Classification"
train = pd.read_csv('dados_treino_so_DUPLA_joaovca3.csv')
test = pd.read_csv('dados_teste_so_DUPLA_joaovca3.csv')

In [13]:
train.head()

,sentence,target
0,(This is an ad hoc connection that differs fro...,HG
1,Online advertising allows businesses to reach ...,AI
2,Here are some of the common security and priva...,AI
3,(In many ways Forth anticipated the principles...,HG
4,is a method used for dynamic web sites/pages t...,HG


In [14]:
test.head()

,sentence,target
0,Today Java applets rep-resent the most widespr...,HG
1,The authentication can occur between client an...,HG
2,These early systems were limited in their accu...,AI
3,BAT extension.,HG
4,Advertisers can track and measure the performa...,AI


___
# Classificador Automático (Boot)


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

___
## Montando SEU Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [31]:
def limpar_mensagens(df, texto_coluna):
    df[texto_coluna] = df[texto_coluna].str.replace(r'[\n:\"\',\(\)]', ' ', regex=True).str.strip() #Remove caracteres desnecessários
    df[texto_coluna] = df[texto_coluna].str.lower() #Converte todos os caracteres para minúsculo
    df[texto_coluna] = df[texto_coluna].str.replace(r'\s+', ' ', regex=True).str.strip() #Remove múltiplos espaços
    df[texto_coluna] = df[texto_coluna].str.replace(r'[^a-zA-Z\s]', '', regex=True) #Remoção de quaisquer caracteres não alfabéticos
    return df


train = limpar_mensagens(train, 'sentence')  # Substitua 'mensagem' pela coluna real do texto no arquivo CSV
test = limpar_mensagens(test, 'sentence')    # Substitua 'mensagem' pela coluna real do texto no arquivo CSV

In [ ]:
from collections import Counter

def suavizacao_laplace(train_df, texto_coluna, categoria_coluna):
    """
    Implementa a suavização de Laplace para calcular probabilidades de palavras por classe,
    usando o método value_counts para contar frequências.

    Parâmetros:
        train_df (DataFrame): Dados de treinamento com frases e classificações.
        texto_coluna (str): Nome da coluna que contém os textos.
        categoria_coluna (str): Nome da coluna que contém as classificações ('HG' ou 'AI').

    Retorno:
        dict: Tabela de probabilidades ajustadas para cada classe.
    """
    # Separar classes do dataset
    classes = train_df[categoria_coluna].unique()  # Ex.: ['HG', 'AI']
    vocabulário = set()  # Contém todas as palavras únicas
    probabilidade_por_classe = {}  # Dicionário para armazenar resultados finais

    # Loop por classe
    for classe in classes:
        # Filtrar textos da classe atual
        textos_classe = train_df[train_df[categoria_coluna] == classe][texto_coluna]

        # Unir todos os textos da classe em uma única string e dividir em palavras
        todas_palavras = ' '.join(textos_classe).split()
        vocabulário.update(todas_palavras)  # Atualiza o vocabulário com as palavras dessa classe

        # Contar frequência de palavras com value_counts()
        contador = pd.Series(todas_palavras).value_counts()
        total_palavras_classe = contador.sum()  # Soma de todas as contagens de palavras

        # Inicializar dicionário para armazenar os cálculos
        probabilidade_palavra = {}

        # Calculando a probabilidade com suavização de Laplace
        for palavra in vocabulário:
            # P(palavra|classe) = (frequência_palavra + 1) / (total_palavras_classe + len(vocabulário))
            probabilidade = (contador.get(palavra, 0) + 1) / (total_palavras_classe + len(vocabulário))
            probabilidade_palavra[palavra] = float(probabilidade)

        # Associar as probabilidades da classe ao dicionário principal
        probabilidade_por_classe[classe] = probabilidade_palavra

    return probabilidade_por_classe

# Aplique suavização de Laplace
resultados = suavizacao_laplace(train, texto_coluna='sentence', categoria_coluna='target')

# Visualize as probabilidades de palavras
print(resultados['HG'])  # Probabilidades Laplace para classe HG
print(resultados['AI'])  # Probabilidades Laplace para classe AI

{'reloaded': np.float64(9.301027763567875e-05), 'flexible': np.float64(0.0001860205552713575), 'grow': np.float64(9.301027763567875e-05), 'usessuch': np.float64(9.301027763567875e-05), 'program': np.float64(0.0021857415244384503), 'since': np.float64(0.0010696181928103055), 'modern': np.float64(0.0004650513881783937), 'needs': np.float64(0.000511556526996233), 'structuring': np.float64(9.301027763567875e-05), 'microcomputers': np.float64(9.301027763567875e-05), 'their': np.float64(0.0017206901362600568), 'maintain': np.float64(0.0001860205552713575), 'information': np.float64(0.00148816444217086), 'applying': np.float64(0.0001860205552713575), 'bold': np.float64(9.301027763567875e-05), 'withvisual': np.float64(9.301027763567875e-05), 'non': np.float64(9.301027763567875e-05), 'rbac': np.float64(9.301027763567875e-05), 'auction': np.float64(0.00013951541645351812), 'extend': np.float64(9.301027763567875e-05), 'wake': np.float64(9.301027763567875e-05), 'generally': np.float64(0.0006975770

___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
## Análise Qualitativa da Performance do Classificador

___
## Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**